<h3> Análisis de migración de la margen de hielo en Nevados de Colombia 2000 - 2022 </h3>
<p><strong>Nombre: </strong> Juan Sebastián Hernández Santana </p>
<p><strong>Fecha: </strong> 06 de Enero de 2024 </p>

In [27]:
# Conexión con la versión desarrollador de Google Earth Engine
import ee
import geemap
# Manejo de datos
import pandas as pd
import numpy as np
# Manejo de gráficos
import matplotlib.pyplot as plt
import seaborn as sns
# Machine learning
from geemap import ml
from sklearn import ensemble

In [32]:
# Funciones
# Enmascaramiento de nubes
def maskL7sr(img):
    cloudShadowBitMask = (1 << 3)
    cloudsBitMask = (1 << 4)
    qa = img.select('QA_PIXEL')
    mask = qa.bitwiseAnd(cloudShadowBitMask).eq(0).And(qa.bitwiseAnd(cloudsBitMask).eq(0))
    return img.updateMask(mask)
# Cortar colección de imágenes
def clip_collection(img):
    return img.clip(nevado).copyProperties(img,["system:time_start"])
# Cálculo de índice glaciar diferencial normalizado NDGI
def NDGI_4_5_7(img):
    ndgi = img.expression('((NIR - GREEN) / (NIR + GREEN))',
    {'NIR': img.select('SR_B4'),
    'GREEN': img.select('SR_B2')}).rename('NDGI').copyProperties(img, ['system:time_start'])
    return img.addBands(ndgi)
def NDGI_8(img):
    ndgi = img.expression('((NIR - GREEN) / (NIR + GREEN))',
    {'NIR': img.select('SR_B5'),
    'GREEN': img.select('SR_B3')}).rename('NDGI').copyProperties(img, ['system:time_start'])
    return img.addBands(ndgi)
# Seleccionar la colección de imágenes según el sensor
def getLandsatImages(sensor):
    if sensor == "L4":
        collection = ee.ImageCollection("LANDSAT/LT04/C02/T1_L2")
    elif sensor == "L5":
        collection = ee.ImageCollection("LANDSAT/LT05/C02/T1_L2")
    elif sensor == "L7":
        collection = ee.ImageCollection("LANDSAT/LE07/C02/T1_L2")
    elif sensor == "L8":
        collection = ee.ImageCollection("LANDSAT/LC08/C02/T1_L2")
    else:
        return None
# Exportar imagen
def exportImageLocal(image, filename):
    task = geemap.ee_export_image(
        image, filename, scale = 30, crs = None, crs_transform = None, region = aoi, dimensions=None,file_per_band=False,
    )

In [33]:
ee.Authenticate()
ee.Initialize()

<p> 1. Área de ínteres (Nevados, Purace) </p>

<p> 1.1. Parámetros de entrada </p>

In [34]:
'''
list_nevados = ['Nevado_Cocuy', 'Nevado_Huila', 'Nevado_Purace', 'Nevado_Nevados']
name_nevado = list_nevados[3] # Seleccionar el nevado de interés
list_year = ['1990', '1995', '2000', '2005', '2010']#, '2015', '2020']

nevado = geemap.shp_to_ee('./Shapefile_Nevados/{}.shp'.format(name_nevado))
'''

# 0: Sierra Nevada de Santa Marta
# 1: Nevado del Cocuy

select_nevado = 0

if select_nevado == 0:
    name_nevado = 'Sierra Nevada de Santa Marta'
    xmin, ymin, xmax, ymax = -73.8336032327945, 10.730927735740165, -73.46047211631704, 10.923352122454183
elif select_nevado == 1:
    name_nevado = 'Nevado del Cocuy'
    xmin, ymin, xmax, ymax = -73.8336032327945, 10.730927735740165, -73.46047211631704, 10.923352122454183

aoi = ee.Geometry.Rectangle([xmin, ymin, xmax, ymax])
nevado = ee.Feature(aoi, {'Nombre': name_nevado})

Map = geemap.Map(basemap = 'HYBRID')
Map.add_basemap('Esri.WorldImagery')

Map.addLayer(nevado, {}, 'Nevado de la Sierra Nevada de Santa Marta')
Map.centerObject(nevado, 10)

In [37]:
def getLandsatImages(sensor):
    if sensor == "L4":
        return ee.ImageCollection("LANDSAT/LT04/C02/T1_L2").filterMetadata('CLOUD_COVER', 'less_than', 70).map(NDGI_4_5_7)
    elif sensor == "L5":
        return ee.ImageCollection("LANDSAT/LT05/C02/T1_L2").filterMetadata('CLOUD_COVER', 'less_than', 70).map(NDGI_4_5_7)
    elif sensor == "L7":
        return ee.ImageCollection("LANDSAT/LE07/C02/T1_L2").filterMetadata('CLOUD_COVER', 'less_than', 70).map(NDGI_4_5_7)
    elif sensor == "L8":
        return ee.ImageCollection("LANDSAT/LC08/C02/T1_L2").filterMetadata('CLOUD_COVER', 'less_than', 70).map(NDGI_8)
    else:
        return None

data = {
    'date_initial': ["1990-11-01", "1995-12-01", "2000-09-01", "2005-12-01", "2010-01-01", "2015-03-07", "2020-12-01", "2023-12-01"],
    'date_end': ["1990-11-30", "1995-12-30", "2000-09-30",  "2005-12-30", "2010-01-30", "2015-03-11", "2020-12-30",  "2023-12-30"],
    'sensor': ["L4", "L5", "L5", "L7", "L7", "L8", "L8", "L8"]
}
df_dates = pd.DataFrame(data)

for i in range(len(df_dates)):
    collection = getLandsatImages(df_dates['sensor'][i])

    if collection is not None:
        images = (
            collection
            .filterDate(df_dates['date_initial'][i], df_dates['date_end'][i])
            # .map(maskL7sr)
            .map(clip_collection)
            .mean()
        )
        
        # Map.addLayer(images.select(["SR_B4", "SR_B5", "SR_B1"]), {}, '{}'.format(df_dates['date_initial'][i]))
        customPalette = ['lightblue', 'blue', 'cyan', 'green', 'yellow', 'orange', 'red']
        Map.addLayer(images.select("NDGI"), {"min": -1, "max": 1, "palette": customPalette}, "NDGI {}".format(df_dates['date_initial'][i]))

        exportImageLocal(images.select("NDGI"), "NDGI_{}_{}.tif".format(df_dates['date_initial'][i], name_nevado))

    else:
        print("Sensor no válido:", df_dates['sensor'][i])


Generating URL ...
Please wait ...
Data downloaded to c:\Users\ingju\OneDrive\Documentos\GitHub\MGITG\GeoAI\ML_Nevados\NDGI_1990-11-01_Sierra Nevada de Santa Marta.tif
Generating URL ...
Please wait ...
Data downloaded to c:\Users\ingju\OneDrive\Documentos\GitHub\MGITG\GeoAI\ML_Nevados\NDGI_1995-12-01_Sierra Nevada de Santa Marta.tif
Generating URL ...
Please wait ...
Data downloaded to c:\Users\ingju\OneDrive\Documentos\GitHub\MGITG\GeoAI\ML_Nevados\NDGI_2000-09-01_Sierra Nevada de Santa Marta.tif
Generating URL ...
Please wait ...
Data downloaded to c:\Users\ingju\OneDrive\Documentos\GitHub\MGITG\GeoAI\ML_Nevados\NDGI_2005-12-01_Sierra Nevada de Santa Marta.tif
Generating URL ...
Please wait ...
Data downloaded to c:\Users\ingju\OneDrive\Documentos\GitHub\MGITG\GeoAI\ML_Nevados\NDGI_2010-01-01_Sierra Nevada de Santa Marta.tif
Generating URL ...
Please wait ...
Data downloaded to c:\Users\ingju\OneDrive\Documentos\GitHub\MGITG\GeoAI\ML_Nevados\NDGI_2015-03-07_Sierra Nevada de Santa Ma

<p> 1.2. Selección de puntos para entrenamiento </p>

In [38]:
PuntoEntrenamiento = geemap.shp_to_ee(r"C:\Users\ingju\OneDrive\Documentos\GitHub\MGITG\GeoAI\ML_Nevados\Shapefile_Muestras\Sample_2023.shp")

In [41]:
def getLandsatImages(sensor):
    if sensor == "L4":
        return ee.ImageCollection("LANDSAT/LT04/C02/T1_L2").filterMetadata('CLOUD_COVER', 'less_than', 70).map(NDGI_4_5_7)
    elif sensor == "L5":
        return ee.ImageCollection("LANDSAT/LT05/C02/T1_L2").filterMetadata('CLOUD_COVER', 'less_than', 70).map(NDGI_4_5_7)
    elif sensor == "L7":
        return ee.ImageCollection("LANDSAT/LE07/C02/T1_L2").filterMetadata('CLOUD_COVER', 'less_than', 70).map(NDGI_4_5_7)
    elif sensor == "L8":
        return ee.ImageCollection("LANDSAT/LC08/C02/T1_L2").filterMetadata('CLOUD_COVER', 'less_than', 70).map(NDGI_8)
    else:
        return None

data = {
    'date_initial': ["1990-11-01", "1995-12-01", "2000-09-01", "2005-12-01", "2010-01-01", "2015-03-07", "2020-12-01", "2023-12-01"],
    'date_end': ["1990-11-30", "1995-12-30", "2000-09-30",  "2005-12-30", "2010-01-30", "2015-03-11", "2020-12-30",  "2023-12-30"],
    'sensor': ["L4", "L5", "L5", "L7", "L7", "L8", "L8", "L8"]
}
df_dates = pd.DataFrame(data)

for i in range(len(df_dates) - 1, len(df_dates) ):
    collection = getLandsatImages(df_dates['sensor'][i])

    if collection is not None:
        images = (
            collection
            .filterDate(df_dates['date_initial'][i], df_dates['date_end'][i])
            # .map(maskL7sr)
            .map(clip_collection)
            .mean()
        )
        
        # Map.addLayer(images.select(["SR_B4", "SR_B5", "SR_B1"]), {}, '{}'.format(df_dates['date_initial'][i]))
        customPalette = ['lightblue', 'blue', 'cyan', 'green', 'yellow', 'orange', 'red']
        Map.addLayer(images.select("NDGI"), {"min": -1, "max": 1, "palette": customPalette}, "NDGI {}".format(df_dates['date_initial'][i]))

        exportImageLocal(images.select("NDGI"), "NDGI_{}_{}.tif".format(df_dates['date_initial'][i], name_nevado))

    else:
        print("Sensor no válido:", df_dates['sensor'][i])

Generating URL ...
Please wait ...
Data downloaded to c:\Users\ingju\OneDrive\Documentos\GitHub\MGITG\GeoAI\ML_Nevados\NDGI_2023-12-01_Sierra Nevada de Santa Marta.tif


In [44]:
bandas = ['SR_B1','SR_B2','SR_B3','SR_B4','SR_B5','SR_B6','SR_B7','NDGI']
etiqueta = "zona"

PuntoEntrenamiento = geemap.shp_to_ee(r"C:\Users\ingju\OneDrive\Documentos\GitHub\MGITG\GeoAI\ML_Nevados\Shapefile_Muestras\Sample_2023.shp")

Muestraentrenamiento = images.select(bandas).sampleRegions(
    **{'collection': PuntoEntrenamiento, 'properties': [etiqueta], 'scale': 30}
)
muestra = Muestraentrenamiento.randomColumn()

muestraEntrenamiento = muestra.filter('random <= 0.8') #Tomamos el 80% de los datos para entrenar el modelo
muestraValidacion = muestra.filter('random > 0.8') #Tomamos el 20% de los datos para validar el modelo

EntrenamientoRF = ee.Classifier.smileRandomForest(**{"numberOfTrees":10}).train(muestraEntrenamiento,etiqueta,bandas)

exactitudEntrenamiento = EntrenamientoRF.confusionMatrix()

validacionDeDatos = muestraValidacion.classify(EntrenamientoRF)

exactitudValidacion = validacionDeDatos.errorMatrix(etiqueta, 'classification')

img_clasificada_2014 = images.select(bandas).classify(EntrenamientoRF)
Map.addLayer(img_clasificada_2014.select("classification"), {}, 'coBerturas 2014 segunda prueba')

In [1]:
display('Resultados del entrenamiento', EntrenamientoRF.explain())

NameError: name 'EntrenamientoRF' is not defined

In [56]:
display('Matriz de confusión del entrenamiento', exactitudEntrenamiento)

'Matriz de confusión del entrenamiento'

In [57]:
display('Exactitud global del entrenamiento', exactitudEntrenamiento.accuracy())

'Exactitud global del entrenamiento'

In [60]:
display('Matriz de confusión de la validacion', exactitudValidacion)

'Matriz de confusión de la validacion'

In [61]:
display('Exactitud de la validación', exactitudValidacion.accuracy())

'Exactitud de la validación'